## Transformer Multimodal (75 Samples)

In [1]:
import os
import re
from scipy.io import loadmat
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

## Extracting data from .s1p files and .mat files

In [2]:
### Helper Functions ###

def extract_s11_from_s1p(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Skip header lines that start with ! or #
            if not line.startswith(('!', '#')):
                columns = line.split()
                # Collect only S11_Real (second column)
                data.append(float(columns[1]))
    return data

# Function to parse Lg, Vds, and Vgs from filenames
def parse_filename_parameters(filename):
    # Initialize default values
    Lg = Vds = Vgs = 'NA'
    
    # Use regex to search for Lg, Vds, and Vgs patterns in the filename
    lg_match = re.search(r'Lg(\d+)p(\d+)', filename)
    vds_match = re.search(r'Vds(\d+)', filename)
    vgs_match = re.search(r'Vgs(\d+)', filename)

    # Process the Lg match with "p" as decimal point
    if lg_match:
        Lg = float(f"{lg_match.group(1)}.{lg_match.group(2)}")
    if vds_match:
        Vds = int(vds_match.group(1))
    if vgs_match:
        Vgs = int(vgs_match.group(1))
    
    # If 'Opend' is in the filename, set Vds to 10000
    if 'opend' in filename:
        Vds = 10000
    
    return Lg, Vds, Vgs

# Function to find repeating pattern
def find_repeating_pattern(waveform, min_period=1000, num_periods=5):
    # Calculate autocorrelation
    autocorr = np.correlate(waveform, waveform, mode='full')
    autocorr = autocorr[autocorr.size // 2:]  # Keep only the second half

    # Find the first peak after lag=0
    differences = np.diff(autocorr)  # Differences between consecutive points
    peaks = np.where((differences[:-1] > 0) & (differences[1:] < 0))[0] + 1  # Peak detection
    
    # Find the period
    if len(peaks) > 0:
        period = peaks[0]  # The first peak indicates the repeating period
        if period < min_period:
            period = min_period  # Enforce minimum period
    else:
        period = min_period  # Default to minimum period if no peaks are found

    # Extract multiple periods of the waveform
    end_index = period * num_periods
    repeating_pattern = waveform[:end_index]
    
    return repeating_pattern, period

def filter_dataset_by_columns(main_dataset, subset_dataset, matching_columns):
    """
    Filters rows in the main dataset where the values in matching columns 
    match those in the subset dataset.

    Parameters:
        main_dataset (pd.DataFrame): The primary dataset to filter.
        subset_dataset (pd.DataFrame): The subset with matching criteria.
        matching_columns (list): List of column names to match on.

    Returns:
        pd.DataFrame: A filtered dataset with matching rows.
    """
    # Filter rows in the main dataset where matching column values are in the subset dataset
    filtered_dataset = main_dataset[
        main_dataset[matching_columns].apply(tuple, axis=1).isin(
            subset_dataset[matching_columns].apply(tuple, axis=1)
        )
    ]
    return filtered_dataset

def smooth_dataframe_columns(df, group_size, fixed_start_cols, fixed_end_cols):
    
    if group_size <= 0:
        raise ValueError("Group size must be greater than 0.")
    
    # Separate fixed columns
    fixed_start = df.iloc[:, :fixed_start_cols]
    fixed_end = df.iloc[:, -fixed_end_cols:]
    
    # Columns to smooth (excluding fixed columns)
    smooth_cols = df.iloc[:, fixed_start_cols:-fixed_end_cols]

    # Smooth by averaging every `group_size` columns
    smoothed_data = []
    for i in range(0, smooth_cols.shape[1], group_size):
        chunk = smooth_cols.iloc[:, i:i+group_size]
        smoothed_data.append(chunk.mean(axis=1))

    # Handle remaining columns if not a perfect multiple of group_size
    if smooth_cols.shape[1] % group_size != 0:
        remaining_cols = smooth_cols.iloc[:, -(smooth_cols.shape[1] % group_size):]
        smoothed_data.append(remaining_cols.mean(axis=1))

    # Combine fixed columns with smoothed data
    smoothed_df = pd.concat([fixed_start] + smoothed_data + [fixed_end], axis=1)
    return smoothed_df

In [3]:
def creating_sync_dataset(S1P_file_path, TDR_file_path):
    
    s11_data = {}
    frequency_data = None
    
    # Loop over all .s1p files in the directory
    for filename in os.listdir(S1P_file_path):
        if filename.endswith('.s1p'):
            file_path = os.path.join(S1P_file_path, filename)
            s11_values = extract_s11_from_s1p(file_path)
            
            # Use the first file's frequency values as a reference
            if frequency_data is None:
                with open(file_path, 'r') as file:
                    frequency_data = [float(line.split()[0]) for line in file if not line.startswith(('!', '#'))]
            
            s11_data[filename] = s11_values
            
    s11_dataset = pd.DataFrame(s11_data).T
    s11_dataset.columns = [f'Frequency_{i}' for i in range(len(s11_dataset.columns))]  # Set column names dynamically
    s11_dataset.insert(0, 'File', s11_dataset.index) 
    
    Duty_list = []

    #for index, row in s11_dataframe.iterrows():
    for i in s11_dataset['File']:
        if 'dut1' in i:
            Duty_list.append(1)
        elif 'dut5' in i:
            Duty_list.append(0)
        else:
            Duty_list.append('NA')
    
    s11_dataset = s11_dataset.drop(columns=['Frequency_0'], errors='ignore')  # Ignore error if column does not exist
    
    s11_dataset[['Lg', 'Vds', 'Vgs']] = s11_dataset['File'].apply(lambda x: pd.Series(parse_filename_parameters(x)))
    s11_dataset = s11_dataset[['Lg', 'Vds', 'Vgs'] + [col for col in s11_dataset.columns if col.startswith('Frequency')]]
    s11_dataset['Duty'] = Duty_list
    
    # Initialize a dictionary to store the loaded data
    mat_data = {}

    # Loop through the files in the folder
    for file_name in os.listdir(TDR_file_path):
        if file_name.endswith('.mat'):  # Check if the file is a .mat file
            # Construct the full file path
            file_path = os.path.join(TDR_file_path, file_name)
            # Load the .mat file and store it in the dictionary
            mat_data[file_name] = loadmat(file_path)

    # Access the loaded data as needed
    tdr_train = mat_data.get('TDR_train.mat')
    tdr_test = mat_data.get('TDR_test.mat')
    tdr_val = mat_data.get('TDR_val.mat')
    
    # Extract and convert the datasets to DataFrames
    train_data = pd.DataFrame(tdr_train["dataTDRtrain"])
    test_data = pd.DataFrame(tdr_test["dataTDRtest"])
    val_data = pd.DataFrame(tdr_val["dataTDRval"])
    
    num_columns = train_data.shape[1]
    column_names = [f"t_{i+1}" for i in range(num_columns)]

    # Assign these column names to the DataFrame
    train_data.columns = column_names
    test_data.columns = column_names
    val_data.columns = column_names
    
    # Specify the path to your Excel file
    excel_file_path = r'C:\Master_thesis\creating_dataset\TDR\key_identifiers.xlsx'

    # Load each sheet into a separate DataFrame
    identifiers_train = pd.read_excel(excel_file_path, sheet_name='train', header=1)
    identifiers_test = pd.read_excel(excel_file_path, sheet_name='test', header=1)
    identifiers_val = pd.read_excel(excel_file_path, sheet_name='val', header=1)

    train_dataset = pd.concat([identifiers_train, train_data], axis=1)
    test_dataset = pd.concat([identifiers_test, test_data], axis=1)
    val_dataset = pd.concat([identifiers_val, val_data], axis=1)

    # The column to move to the last position (for example, column 'B')
    col_to_move = 'Duty'

    # Function to move a column to the last position in a DataFrame
    def move_column_to_last(df, col_to_move):
        cols = [col for col in df.columns if col != col_to_move]
        df = df[cols + [col_to_move]]
        return df

    # Apply the function to each dataset
    train_dataset = move_column_to_last(train_dataset, col_to_move)
    test_dataset = move_column_to_last(test_dataset, col_to_move)
    val_dataset = move_column_to_last(val_dataset, col_to_move)

    TDR_dataset = pd.concat([train_dataset, test_dataset, val_dataset], axis=0, ignore_index=True)

    TDR_dataset['Duty'] = TDR_dataset['Duty'].replace({1: 1, 5: 0})
    print()
    # Columns for comparison
    common_columns = ['Lg', 'Vds', 'Vgs', 'Duty']

    # Identify common samples
    common_samples = pd.merge(s11_dataset[common_columns], TDR_dataset[common_columns], on=common_columns)

    # Subset 1: Rows in df1 not in common samples
    #unsync_s11_dataset = s11_dataset[~s11_dataset[common_columns].apply(tuple, axis=1).isin(common_samples.apply(tuple, axis=1))]
    
    # Subset 2: Common samples from df1 with all df1 columns
    sync_s11_dataset = s11_dataset[s11_dataset[common_columns].apply(tuple, axis=1).isin(common_samples.apply(tuple, axis=1))]
    #print(sync_s11_dataset.shape)
    # Subset 3: Common samples from df2 with all df2 columns
    sync_TDR_dataset = TDR_dataset[TDR_dataset[common_columns].apply(tuple, axis=1).isin(common_samples.apply(tuple, axis=1))]

    # Subset 4: Rows in df2 not in common samples
    unsync_TDR_dataset = TDR_dataset[~TDR_dataset[common_columns].apply(tuple, axis=1).isin(common_samples.apply(tuple, axis=1))]

    sync_TDR_dataset = smooth_dataframe_columns(sync_TDR_dataset, 63, 3, 1)
    unsync_TDR_dataset = smooth_dataframe_columns(unsync_TDR_dataset, 63, 3, 1)
    TDR_dataset = smooth_dataframe_columns(TDR_dataset, 63, 3, 1)
    
    # Create a DataFrame filled with zeros
    unsync_s11_dataset = pd.DataFrame(np.zeros((45, 1004)))
    
    # Rename only the last column
    col_names = [f"col_{i}" for i in range(unsync_s11_dataset.shape[1])]  # Generate default column names
    col_names[-1] = "Duty" 
    col_names[0] = "Lg" 
    col_names[1] = "Vds" 
    col_names[2] = "Vgs"
    unsync_s11_dataset.columns = col_names
    
    unsync_s11_dataset['Duty'] = unsync_TDR_dataset['Duty'].values
    unsync_s11_dataset['Lg'] = unsync_TDR_dataset['Lg'].values
    unsync_s11_dataset['Vds'] = unsync_TDR_dataset['Vds'].values
    unsync_s11_dataset['Vgs'] = unsync_TDR_dataset['Vgs'].values
    
    
    for i in range (3, len(unsync_s11_dataset.columns)-1):
        col_name = f'Frequency_{i-2}'
        unsync_s11_dataset.columns.values[i] = col_name

    s11_dataset = pd.concat([sync_s11_dataset, unsync_s11_dataset], ignore_index=True)
    
    return s11_dataset, TDR_dataset

In [4]:
S1P_file_path = r'C:\Master_thesis\creating_dataset\Dataset\S11'
TDR_file_path = r'C:\Master_thesis\creating_dataset\TDR'

In [5]:
s11_dataset, TDR_dataset = creating_sync_dataset(S1P_file_path, TDR_file_path)

In [6]:
s11_dataset.shape, TDR_dataset.shape

((75, 1004), (75, 1006))

In [7]:
def data_Scaling_Sync(sync_s11_dataset, sync_TDR_dataset, scaling_type):
    
    train_shape = int((sync_s11_dataset.shape[0])*0.6)
    test_shape = int((sync_s11_dataset.shape[0])*0.2)
    val_shape = int((sync_s11_dataset.shape[0])*0.2)

    sync_s11_dataset = sync_s11_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    # Step 1: Split into train and temp_data (test + val)
    s11_train, temp_data = train_test_split(sync_s11_dataset, train_size=train_shape, random_state=42, stratify=sync_s11_dataset["Duty"])

    # Step 2: Split temp_data into test and val (ensuring balance in "Duty")
    s11_test, s11_val = train_test_split(temp_data, train_size=test_shape, random_state=42, stratify=temp_data["Duty"])

    # Columns for matching
    matching_column = ['Lg', 'Vds', 'Vgs', 'Duty']

    TDR_train = filter_dataset_by_columns(sync_TDR_dataset, s11_train, matching_column)
    TDR_test = filter_dataset_by_columns(sync_TDR_dataset, s11_test, matching_column)
    TDR_val = filter_dataset_by_columns(sync_TDR_dataset, s11_val, matching_column)
    
    # Perform an inner merge to keep only rows where both A and B match
    merged_train = pd.merge(s11_train, TDR_train, on=['Lg', 'Vds', 'Vgs', 'Duty'], how='inner')
    merged_test = pd.merge(s11_test, TDR_test, on=['Lg', 'Vds', 'Vgs', 'Duty'], how='inner')
    merged_val = pd.merge(s11_val, TDR_val, on=['Lg', 'Vds', 'Vgs', 'Duty'], how='inner')
    
    # Separate the DataFrames back
    s11_train = merged_train[s11_train.columns]
    TDR_train = merged_train[TDR_train.columns]
    
    s11_test = merged_test[s11_test.columns]
    TDR_test = merged_test[TDR_test.columns]
    
    s11_val = merged_val[s11_val.columns]
    TDR_val = merged_val[TDR_val.columns]
       
    
    #TDR_train = smooth_dataframe_columns(TDR_train, 63, 3, 1)
    #TDR_test = smooth_dataframe_columns(TDR_test, 63, 3, 1)
    #TDR_val = smooth_dataframe_columns(TDR_val, 63, 3, 1)
    
    X_s11_train = s11_train.iloc[:, 0:-1].values
    y_s11_train = s11_train['Duty'].values

    X_s11_test = s11_test.iloc[:, 0:-1].values
    y_s11_test = s11_test['Duty'].values

    X_s11_val = s11_val.iloc[:, 0:-1].values
    y_s11_val = s11_val['Duty'].values

    X_TDR_train = TDR_train.iloc[:, 0:-1].values
    y_TDR_train = TDR_train['Duty'].values

    X_TDR_test = TDR_test.iloc[:, 0:-1].values
    y_TDR_test = TDR_test['Duty'].values

    X_TDR_val = TDR_val.iloc[:, 0:-1].values
    y_TDR_val = TDR_val['Duty'].values
    
    # Mapping of scaling types to scaler objects
    scalers = {
        'standard': StandardScaler(),
        'minmax': MinMaxScaler(),
        'robust': RobustScaler(),
        'maxabs': MaxAbsScaler()
    }

    # Check if the scaling_type is valid
    if scaling_type not in scalers:
        raise ValueError(f"Invalid scaling_type. Choose from {list(scalers.keys())}.")
    
    # Select the appropriate scaler
    scaler = scalers[scaling_type] 
    
    X_s11_train_scaled = scaler.fit_transform(X_s11_train)
    X_s11_test_scaled = scaler.transform(X_s11_test)
    X_s11_val_scaled = scaler.transform(X_s11_val)
        
    X_TDR_train_scaled = scaler.fit_transform(X_TDR_train)
    X_TDR_test_scaled = scaler.transform(X_TDR_test)
    X_TDR_val_scaled = scaler.transform(X_TDR_val)
    
    # Convert data to PyTorch tensors
    X_s11_train_scaled = torch.tensor(X_s11_train_scaled, dtype=torch.float32)
    y_s11_train = torch.tensor(y_s11_train, dtype=torch.long)
    X_s11_test_scaled = torch.tensor(X_s11_test_scaled, dtype=torch.float32)
    y_s11_test = torch.tensor(y_s11_test, dtype=torch.long)
    X_s11_val_scaled = torch.tensor(X_s11_val_scaled, dtype=torch.float32)
    y_s11_val = torch.tensor(y_s11_val, dtype=torch.long)

    # Convert data and labels to TensorDatasets and create DataLoaders
    s11_train_dataset = TensorDataset(X_s11_train_scaled, y_s11_train.long())
    s11_val_dataset = TensorDataset(X_s11_val_scaled, y_s11_val.long())
    s11_test_dataset = TensorDataset(X_s11_test_scaled, y_s11_test.long())

    s11_train_loader = torch.utils.data.DataLoader(s11_train_dataset, batch_size=4, shuffle=True)
    s11_test_loader = torch.utils.data.DataLoader(s11_test_dataset, batch_size=4)
    s11_val_loader = torch.utils.data.DataLoader(s11_val_dataset, batch_size=4)

    # Convert data to PyTorch tensors
    X_TDR_train_scaled = torch.tensor(X_TDR_train_scaled, dtype=torch.float32)
    y_TDR_train = torch.tensor(y_TDR_train, dtype=torch.long)
    X_TDR_test_scaled = torch.tensor(X_TDR_test_scaled, dtype=torch.float32)
    y_TDR_test = torch.tensor(y_TDR_test, dtype=torch.long)
    X_TDR_val_scaled = torch.tensor(X_TDR_val_scaled, dtype=torch.float32)
    y_TDR_val = torch.tensor(y_TDR_val, dtype=torch.long)

    # Convert data and labels to TensorDatasets and create DataLoaders
    TDR_train_dataset = TensorDataset(X_TDR_train_scaled, y_TDR_train.long())
    TDR_val_dataset = TensorDataset(X_TDR_val_scaled, y_TDR_val.long())
    TDR_test_dataset = TensorDataset(X_TDR_test_scaled, y_TDR_test.long())

    TDR_train_loader = torch.utils.data.DataLoader(TDR_train_dataset, batch_size=4, shuffle=True)
    TDR_test_loader = torch.utils.data.DataLoader(TDR_test_dataset, batch_size=4)
    TDR_val_loader = torch.utils.data.DataLoader(TDR_val_dataset, batch_size=4) 
    
    return s11_train_loader, s11_test_loader, s11_val_loader, TDR_train_loader, TDR_test_loader, TDR_val_loader

## Transformer Model

In [8]:
# Transformer Model for Duty Classification

class DutyClassifier(nn.Module):
    def __init__(self, seq_len1, seq_len2, input_dim, num_classes, d_model=128, nhead=8, num_layers=1):
        super(DutyClassifier, self).__init__()
        
        # Separate embedding layers to map input dimensions to d_model
        self.embedding1 = nn.Linear(input_dim, d_model)
        self.embedding2 = nn.Linear(input_dim, d_model)
        
        # Separate transformer encoders for each input
        encoder_layer1 = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        encoder_layer2 = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        
        self.transformer_encoder1 = nn.TransformerEncoder(encoder_layer1, num_layers=num_layers)
        self.transformer_encoder2 = nn.TransformerEncoder(encoder_layer2, num_layers=num_layers)
        
        # Classification layer
        # The total dimension is (seq_len1 + seq_len2) * d_model after concatenation

        self.classifier = nn.Linear(d_model * (seq_len1 + seq_len2), num_classes)
     
    def forward(self, x1, x2):
        # Project each dataset to the common feature dimension (d_model)
        x1 = self.embedding1(x1)  # Shape: [batch_size, seq_len1, d_model]
        x2 = self.embedding2(x2)  # Shape: [batch_size, seq_len2, d_model]

        # Transform the sequence for each dataset
        x1 = x1.permute(1, 0, 2)  # Shape: [seq_len1, batch_size, d_model]
        x1 = self.transformer_encoder1(x1)
        x1 = x1.permute(1, 0, 2)  # Back to [batch_size, seq_len1, d_model]

        x2 = x2.permute(1, 0, 2)  # Shape: [seq_len2, batch_size, d_model]
        x2 = self.transformer_encoder2(x2)
        x2 = x2.permute(1, 0, 2)  # Back to [batch_size, seq_len2, d_model]

        # Concatenate along the sequence length dimension
        x = torch.cat((x1, x2), dim=1)  # Shape: [batch_size, seq_len1 + seq_len2, d_model]

        # Flatten for classification
        x = x.flatten(start_dim=1)  # Shape: [batch_size, (seq_len1 + seq_len2) * d_model]
        out = self.classifier(x)  # Shape: [batch_size, num_classes]

        return out
        

## Training and testing of model

In [9]:
# Training function
def train_model(model, num_epochs, patience):
    best_val_loss = float('inf')
    epochs_no_improve = 0

    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        train_targets, train_preds = [], []
        
        # Iterate over both loaders simultaneously (synchronized)
        for (s11_train_data, train_labels), (tdr_train_data, _) in zip(s11_train_loader, TDR_train_loader):
            # Move inputs and labels to the appropriate device
            s11_train_data = s11_train_data.to(device)
            tdr_train_data = tdr_train_data.to(device)
            train_labels = train_labels.to(device)

            optimizer.zero_grad()
            outputs = model(s11_train_data.unsqueeze(-1), tdr_train_data.unsqueeze(-1))
            loss = criterion(outputs, train_labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            
            train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_targets.extend(train_labels.cpu().numpy())
                
        train_accuracy = accuracy_score(train_targets, train_preds)
        
        # Validation
        model.eval()
        val_loss = 0
        val_preds, val_targets = [], []
        with torch.no_grad():
            
            for (s11_val_data, val_labels), (tdr_val_data, _) in zip(s11_val_loader, TDR_val_loader):
                # Move inputs and labels to the appropriate device
                s11_val_data = s11_val_data.to(device)
                tdr_val_data = tdr_val_data.to(device)
                val_labels = val_labels.to(device)
                
                
                outputs = model(s11_val_data.unsqueeze(-1), tdr_val_data.unsqueeze(-1))
                loss = criterion(outputs, val_labels)
                val_loss += loss.item()

                val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_targets.extend(val_labels.cpu().numpy())

        val_accuracy = accuracy_score(val_targets, val_preds)
        
        # Early stopping logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch + 1} epochs. Best Val Loss: {best_val_loss:.4f}")
            print(f"Epoch {epoch + 1}/{num_epochs}, Train Accuracy: {train_accuracy:.4f}, Val Accuracy: {val_accuracy:.4f}")
            break
            
# Testing function
def test_model(model):
    model.eval()
    test_preds, test_targets = [], []
    with torch.no_grad():
        
        for (s11_test_data, test_labels), (tdr_test_data, _) in zip(s11_test_loader, TDR_test_loader):
            # Move inputs and labels to the appropriate device
            s11_test_data = s11_test_data.to(device)
            tdr_test_data = tdr_test_data.to(device)
            test_labels = test_labels.to(device)
            
            outputs = model(s11_test_data.unsqueeze(-1), tdr_test_data.unsqueeze(-1))
            test_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            test_targets.extend(test_labels.cpu().numpy())

    test_accuracy = accuracy_score(test_targets, test_preds)
    test_conf_matrix = confusion_matrix(test_targets, test_preds)
    classi_report = classification_report(test_targets, test_preds)
    print(test_conf_matrix)
    print('Confusion Matrix')
    print(confusion_matrix)
    print('Test Accuracy - ',np.round(test_accuracy,3))
    print('Classification Report')
    print(classi_report)
     
    return test_accuracy

In [13]:
Multimodal_classifier = DutyClassifier(seq_len1=1003, seq_len2=1005, input_dim=1, num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Multimodal_classifier.parameters(), lr=0.0001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Multimodal_classifier = Multimodal_classifier.to(device)

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [14]:
accuracy_list = []
for i in range(1, 11):
    print('-------------------------------------------------')
    print(f'{i}th Training Loop')
    s11_train_loader, s11_test_loader, s11_val_loader, TDR_train_loader, TDR_test_loader, TDR_val_loader = data_Scaling_Sync(s11_dataset, TDR_dataset, scaling_type='standard')
    train_model(Multimodal_classifier, num_epochs=100, patience=15)
    test_preds = test_model(Multimodal_classifier)
    accuracy_list.append(test_preds)
    
    torch.save(Multimodal_classifier.state_dict(), rf'C:\Master_thesis\creating_dataset\Model\Multimodal_classifier_{i}.pth')
    
    print('-------------------------------------------------')
    i+=1

-------------------------------------------------
1th Training Loop
Early stopping triggered after 20 epochs. Best Val Loss: 3.9308
Epoch 20/100, Train Accuracy: 0.4667, Val Accuracy: 0.6000
[[3 4]
 [2 6]]
Confusion Matrix
<function confusion_matrix at 0x0000024A915831A0>
Test Accuracy -  0.6
Classification Report
              precision    recall  f1-score   support

           0       0.60      0.43      0.50         7
           1       0.60      0.75      0.67         8

    accuracy                           0.60        15
   macro avg       0.60      0.59      0.58        15
weighted avg       0.60      0.60      0.59        15

-------------------------------------------------
-------------------------------------------------
2th Training Loop
Early stopping triggered after 28 epochs. Best Val Loss: 5.2025
Epoch 28/100, Train Accuracy: 0.6889, Val Accuracy: 0.6000
[[6 1]
 [4 4]]
Confusion Matrix
<function confusion_matrix at 0x0000024A915831A0>
Test Accuracy -  0.667
Classificat

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

Early stopping triggered after 22 epochs. Best Val Loss: 7.7275
Epoch 22/100, Train Accuracy: 0.6667, Val Accuracy: 0.6000
[[6 1]
 [5 3]]
Confusion Matrix
<function confusion_matrix at 0x0000024A915831A0>
Test Accuracy -  0.6
Classification Report
              precision    recall  f1-score   support

           0       0.55      0.86      0.67         7
           1       0.75      0.38      0.50         8

    accuracy                           0.60        15
   macro avg       0.65      0.62      0.58        15
weighted avg       0.65      0.60      0.58        15

-------------------------------------------------
-------------------------------------------------
5th Training Loop
Early stopping triggered after 60 epochs. Best Val Loss: 5.8943
Epoch 60/100, Train Accuracy: 0.5111, Val Accuracy: 0.6667
[[0 7]
 [0 8]]
Confusion Matrix
<function confusion_matrix at 0x0000024A915831A0>
Test Accuracy -  0.533
Classification Report
              precision    recall  f1-score   support

  

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

Early stopping triggered after 28 epochs. Best Val Loss: 4.5777
Epoch 28/100, Train Accuracy: 0.6000, Val Accuracy: 0.6000
[[0 7]
 [0 8]]
Confusion Matrix
<function confusion_matrix at 0x0000024A915831A0>
Test Accuracy -  0.533
Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.53      1.00      0.70         8

    accuracy                           0.53        15
   macro avg       0.27      0.50      0.35        15
weighted avg       0.28      0.53      0.37        15

-------------------------------------------------
-------------------------------------------------
7th Training Loop


...

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

Early stopping triggered after 45 epochs. Best Val Loss: 5.2191
Epoch 45/100, Train Accuracy: 0.7111, Val Accuracy: 0.6000
[[0 7]
 [0 8]]
Confusion Matrix
<function confusion_matrix at 0x0000024A915831A0>
Test Accuracy -  0.533
Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.53      1.00      0.70         8

    accuracy                           0.53        15
   macro avg       0.27      0.50      0.35        15
weighted avg       0.28      0.53      0.37        15

-------------------------------------------------
-------------------------------------------------
10th Training Loop


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

Early stopping triggered after 16 epochs. Best Val Loss: 6.3390
Epoch 16/100, Train Accuracy: 0.7333, Val Accuracy: 0.6667
[[0 7]
 [0 8]]
Confusion Matrix
<function confusion_matrix at 0x0000024A915831A0>
Test Accuracy -  0.533
Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.53      1.00      0.70         8

    accuracy                           0.53        15
   macro avg       0.27      0.50      0.35        15
weighted avg       0.28      0.53      0.37        15

-------------------------------------------------


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

In [15]:
accuracy_list

[0.6,
 0.6666666666666666,
 0.5333333333333333,
 0.6,
 0.5333333333333333,
 0.5333333333333333,
 0.5333333333333333,
 0.5333333333333333,
 0.5333333333333333,
 0.5333333333333333]

In [17]:
np.mean(accuracy_list)

np.float64(0.5599999999999999)